In [4]:
import pandas as pd
import vectorbt as vbt

In [5]:
stocks=['AAPL','MSFT','AMZN','GOOG','IBM']
start = '2014-01-01 UTC'  # crypto is in UTC

end = '2020-01-01 UTC'

In [8]:
data=[]
for stock in stocks:
    df=(vbt.YFData.download(stock,start=start,end=end).get('Close'))
    df.name=stock
    data.append(df)
prices=pd.concat(data,axis=1)



In [10]:
prices

,AAPL,MSFT,AMZN,GOOG,IBM
Date,,,,,
2014-01-02 05:00:00+00:00,17.318729,31.290817,19.898500,27.724083,116.571327
2014-01-03 05:00:00+00:00,16.938307,31.080305,19.822001,27.521841,117.268776
2014-01-06 05:00:00+00:00,17.030670,30.423494,19.681499,27.828691,116.866631
2014-01-07 05:00:00+00:00,16.908878,30.659275,19.901501,28.365179,119.197685
2014-01-08 05:00:00+00:00,17.015959,30.111944,20.096001,28.424210,118.104439
...,...,...,...,...,...
2019-12-24 05:00:00+00:00,69.236229,151.657349,89.460503,67.178001,105.961365
2019-12-26 05:00:00+00:00,70.609917,152.900436,93.438499,68.019997,105.906403
2019-12-27 05:00:00+00:00,70.583115,153.179871,93.489998,67.594498,106.189011


In [11]:
# Resetting the index to make 'Date' a column
df_long = prices.reset_index()

# Melting the DataFrame
df_long = df_long.melt(id_vars=['Date'], var_name='ticker', value_name='prc')

# You might want to sort the DataFrame for better organization
df_long = df_long.sort_values(by=['Date', 'ticker'])

In [24]:
df_long

,date,ticker,PRICE
0,2014-01-02 05:00:00,AAPL,17.318729
3020,2014-01-02 05:00:00,AMZN,19.898500
4530,2014-01-02 05:00:00,GOOG,27.724083
6040,2014-01-02 05:00:00,IBM,116.571327
1510,2014-01-02 05:00:00,MSFT,31.290817
...,...,...,...
1509,2019-12-31 05:00:00,AAPL,71.520828
4529,2019-12-31 05:00:00,AMZN,92.391998
6039,2019-12-31 05:00:00,GOOG,66.850998
7549,2019-12-31 05:00:00,IBM,105.223457


In [26]:
import pandas as pd
from data_helper_functions import *
# Assuming df_long is your DataFrame
df_long.rename(columns={'prc': 'PRICE'}, inplace=True)
df_long.rename(columns={'Date':'date'},inplace=True)
df_long.rename(columns={'ticker':'TICKER'},inplace=True)
df_long['date'] = df_long['date'].dt.tz_localize(None)

# Parameters for study periods
window_size = 240
trade_size = 250
train_size = 750
forward_roll = 250
start_date = pd.to_datetime("2014-01-01")
end_date = pd.to_datetime("2019-12-31")
target_type = 'direction'
standardize = True
data_type = 'PRICE'
apply_wavelet_transform = False

# Creating study periods
study_periods = create_study_periods(df_long, window_size, trade_size, train_size, forward_roll, start_date, end_date, target_type, standardize, data_type, apply_wavelet_transform)

# Parameters for tensor creation
n_jobs = 6
sequence_length = 240

# Creating tensors
# Parameters for tensor creation
n_jobs = 6
sequence_length = 240

# Creating tensors
train_test_splits, task_types = create_tensors(study_periods, n_jobs, sequence_length)


 56%|█████▌    | 5/9 [00:00<00:00, 162.02it/s]

Reached the end of the dataset.



[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   2 out of   5 | elapsed:    1.6s remaining:    2.4s
[Parallel(n_jobs=6)]: Done   3 out of   5 | elapsed:    1.6s remaining:    1.1s
[Parallel(n_jobs=6)]: Done   5 out of   5 | elapsed:    1.6s finished
